In [13]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train_data = datasets.FashionMNIST(
    root = "tutorial",
    train = True,
    download = True,
    transform = ToTensor()
)
train_data
train_data.data.shape
np.unique(train_data.targets, return_counts = True)

In [ ]:
test_data = datasets.FashionMNIST(
    root = "tutorial",
    train = False,
    download = True,
    transform = ToTensor()
)
test_data
test_data.data.shape
np.unique(test_data.targets, return_counts = True)

In [ ]:
train_dataloader = DataLoader(dataset = train_data, batch_size = 64)
test_dataloader = DataLoader(dataset = test_data, batch_size = 64)

train_dataloader.dataset
len(train_dataloader.dataset)

len(train_dataloader)

938 * 64

for batch, (x, y) in enumerate(train_dataloader):
    print(batch + 1)
    print(f"shape of x: {x.shape}")
    print(f"shape of y: {y.shape} & dtype of y: {y.dtype}")

In [ ]:
class NET(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features = 28*28,
            out_features = 512),
            nn.ReLU(),
            nn.Linear(in_features = 512,
            out_features = 512),
            nn.ReLU(),
            nn.Linear(in_features = 512,
            out_features = 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        network = self.linear_relu_stack(x)
        return network

simpleNN = NET()
simpleNN

In [27]:
loss_func = nn.CrossEntropyLoss()
opt = torch.optim.SGD(
    params = simpleNN.parameters(),
    lr = 1e-3
)

In [28]:
def training(dataloader, model, loss_func, optimizer):

    size = len(dataloader.dataset)

    model.train()

    for batch, (x, y) in enumerate(dataloader):

        pred = model(x)
        loss = loss_func(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss = loss.item()
            current = batch * len(x)
            print(f"loss: {loss:>5f} at {current:>5d}/{size:>5d}")

In [34]:
def testing(dataloader, model, loss_func):

    size = len(dataloader.dataset)
    num_batch = len(dataloader)

    model.eval()

    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)

            test_loss += loss_func(pred, y).item()
            correct += (pred.argmax (1) == y).type(torch.float).sum().item()

    test_loss /= num_batch
    correct /= size

    print(f"\n test evalution: \n Accuracy: {(100*correct):>5f}%, \
        Avg loss: {test_loss:>5f}\n")


In [ ]:
epochs = 5

for t in range(epochs):
    print(f"\n\n   Epoch {t+1} --------------------- \n")

    training(train_dataloader, simpleNN, loss_func, opt)
    testing(test_dataloader, simpleNN, loss_func)

In [ ]:
from operator import truth

labels = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dres",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

plt.imshow(test_data.data[2], cmap = "Greys")

x = test_data[0][0]

y = test_data[0][1]

with torch.no_grad():
    pred = simpleNN(x)
    prediction = labels[pred[0].argmax(0)]
    truth = labels[y]

    print(f"simpleNN predcit as {prediction} : truth is {truth}")